In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

In [3]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Program Files\Python310\python.exe -m pip install --upgrade pip


In [4]:
!pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 38.0/38.0 MB 10.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Program Files\Python310\python.exe -m pip install --upgrade pip


In [6]:
C:\\Program Files\\Python310\\python.exe -m pip install --upgrade pip

SyntaxError: unexpected character after line continuation character (2679201871.py, line 1)